In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
from pathlib import Path
import csv
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tqdm import tqdm


In [2]:
all_inputs = [
    f
    for f in
    Path('/kaggle/input/waveform-inversion/train_samples').rglob('*.npy')
    if ('seis' in f.stem) or ('data' in f.stem)
]

def inputs_files_to_output_files(input_files):
    return [
        Path(str(f).replace('seis', 'vel').replace('data', 'model'))
        for f in input_files
    ]

all_outputs = inputs_files_to_output_files(all_inputs)

assert all(f.exists() for f in all_outputs)

train_inputs = [all_inputs[i] for i in range(0, len(all_inputs), 2)] # Sample every two
valid_inputs = [f for f in all_inputs if not f in train_inputs]

train_outputs = inputs_files_to_output_files(train_inputs)
valid_outputs = inputs_files_to_output_files(valid_inputs)

def get_global_min_max(label_files):
    mins, maxs = [], []
    for f in tqdm(label_files, desc="scanning label files"):
        arr = np.load(f, mmap_mode="r")
        mins.append(arr.min())
        maxs.append(arr.max())
    return float(np.min(mins)), float(np.max(maxs))

label_min, label_max = get_global_min_max(train_outputs)
print(f"Global label_min={label_min:.2f} | label_max={label_max:.2f}")

class SeismicDataset(Dataset):
    def __init__(self, inputs_files, output_files, n_examples_per_file=500,label_min=None, label_max=None):
        assert len(inputs_files) == len(output_files)
        self.inputs_files = inputs_files
        self.output_files = output_files
        self.n_examples_per_file = n_examples_per_file
        self.label_min = label_min
        self.label_scale = label_max - label_min

    def __len__(self):
        return len(self.inputs_files) * self.n_examples_per_file

    def __getitem__(self, idx):
        # Calculate file offset and sample offset within file
        file_idx = idx // self.n_examples_per_file
        sample_idx = idx % self.n_examples_per_file

        X = np.load(self.inputs_files[file_idx], mmap_mode='r')
        y = np.load(self.output_files[file_idx], mmap_mode='r')

        # map label to [-1,1] 
        y = 2.0 * (y - self.label_min) / self.label_scale - 1.0

        try:
            return X[sample_idx].copy(), y[sample_idx].copy()
        finally:
            del X, y

scanning label files: 100%|██████████| 10/10 [00:01<00:00,  7.47it/s]

Global label_min=1500.00 | label_max=4500.00


In [3]:
dstrain = SeismicDataset(train_inputs, train_outputs,label_min=label_min, label_max=label_max)
dsvalid = SeismicDataset(valid_inputs, valid_outputs,label_min=label_min, label_max=label_max)

train_loader = DataLoader(dstrain, batch_size=8, shuffle=True, num_workers=4)
val_loader = DataLoader(dsvalid, batch_size=8, shuffle=True, num_workers=4)

In [4]:
for X_batch, y_batch in train_loader:
    print("Batch input shape:", X_batch.shape)
    print("Batch target shape:", y_batch.shape)
    break  # just check the first batch


Batch input shape: torch.Size([8, 5, 1000, 70])
Batch target shape: torch.Size([8, 1, 70, 70])


In [5]:
class MLP(nn.Module):
    """Simple 2-layer MLP with GELU and dropout"""
    def __init__(self, dim, hidden_dim, drop=0.):
        super().__init__()
        self.fc1   = nn.Linear(dim, hidden_dim)
        self.act   = nn.GELU()
        self.fc2   = nn.Linear(hidden_dim, dim)
        self.drop  = nn.Dropout(drop)

    def forward(self, x):
        x = self.drop(self.act(self.fc1(x)))
        x = self.drop(self.fc2(x))
        return x

class TransformerBlock(nn.Module):
    """
    ViT-style encoder block
    LayerNorm  →  Multi-Head Self-Attention  →  residual
    LayerNorm  →  MLP                        →  residual
    """
    def __init__(self, dim, num_heads, mlp_ratio=4., drop=0., attn_drop=0.):
        super().__init__()
        self.norm1 = nn.LayerNorm(dim)
        self.attn  = nn.MultiheadAttention(
            embed_dim=dim, num_heads=num_heads,
            dropout=attn_drop, batch_first=True)
        self.drop1 = nn.Dropout(drop)

        self.norm2 = nn.LayerNorm(dim)
        hidden_dim = int(dim * mlp_ratio)
        self.mlp   = MLP(dim, hidden_dim, drop)

    def forward(self, x):
        # x shape: (B, N, D)
        # --- self-attention ---
        attn_out, _ = self.attn(self.norm1(x), self.norm1(x), self.norm1(x))
        x = x + self.drop1(attn_out)

        # --- feed-forward ---
        x = x + self.mlp(self.norm2(x))
        return x


In [6]:
class SeismicViT(nn.Module):
    def __init__(self,
                 in_ch=5, patch=(25,5), embed_dim=256,
                 depth=10, num_heads=8, mlp_ratio=4.,
                 out_size=(70,70)):
        super().__init__()

        # 1) patchify
        self.patch_embed = nn.Conv2d(in_ch, embed_dim,
                                     kernel_size=patch, stride=patch) # B,embed_dim,40,14
        num_patches = (1000 // patch[0]) * (70 // patch[1]) # 40 * 14 = 560
        self.pos = nn.Parameter(torch.zeros(1, num_patches, embed_dim)) # (1, 560, 256)

        # 2) Transformer encoder
        self.blocks = nn.ModuleList([
            TransformerBlock(embed_dim, num_heads, mlp_ratio)
            for _ in range(depth)
        ])
        self.norm = nn.LayerNorm(embed_dim)

        # 3) linear projector to patch-level velocity
        self.head = nn.Linear(embed_dim, patch[0]*patch[1])

        # 4) light decoder to 1000×70
        self.up = nn.Sequential(
            nn.ConvTranspose2d(1, 32, 4, 2, 1),  # 125→250
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose2d(32, 16, 4, 2, 1), # 250→500
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose2d(16,  1, 4, 2, 1), # 500→1000
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(1, 1, kernel_size=3, padding=1)  # 1000×70 → 1000×70
        )
    def forward(self, x):                        # (B,5,1000,70)
        x = self.patch_embed(x).flatten(2).permute(0,2,1)  # -> (B, 256, 560) -> (B, 560, 256)   # (batch, tokens, embed_dim)
        x = x + self.pos  #(B, 560, 256) 
        for blk in self.blocks:
            x = blk(x)
        x = self.norm(x)
        x = self.head(x)                         # (B,N,PatchArea) (B, 560, 125)
        x = x.view(-1, 1, 1000, 70)             # (B,1,1000,70)
        x = F.avg_pool2d(x, kernel_size=(8,1))   # 1000→125   keep 70 (B, 1, 125, 70)
        assert x.shape[1:] == (1, 125, 70), f"Expected (B,1,125,70) but got {x.shape}"
        x = self.up(x)                        # (B,1,1000,70)
        return torch.tanh(F.interpolate(x, size=(70,70), mode='bilinear', align_corners=False)) # interpolate to (70*70)


In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

model = SeismicViT().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
loss_fn = nn.MSELoss()

epochs = 20

best_val_loss = float('inf')

for epoch in range(epochs):
    # ── 1. TRAIN ───────────────────────────────────────────────────────
    model.train()
    running_loss = 0.0
    for xb, yb in tqdm(train_loader, desc=f"[Train] Epoch {epoch+1}"):
        xb, yb = xb.to(device), yb.to(device).squeeze(1)
        optimizer.zero_grad()
        preds = model(xb).squeeze(1)
        loss  = loss_fn(preds, yb)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    train_loss = running_loss / len(train_loader)

    # ── 2. VALIDATE ────────────────────────────────────────────────────
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device).squeeze(1)
            preds  = model(xb).squeeze(1)
            val_loss += loss_fn(preds, yb).item()

    val_loss /= len(val_loader)
    print(f"Epoch {epoch+1} | train {train_loss:.4f} | val {val_loss:.4f}")

    # ── 3. CHECKPOINT ON VAL LOSS ─────────────────────────────────────
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), '/kaggle/working/best_model.pth')
        print(f"✅  New best model saved (val loss {best_val_loss:.4f})")

Using device: cuda


[Train] Epoch 1: 100%|██████████| 625/625 [02:11<00:00,  4.75it/s]


Epoch 1 | train 0.2494 | val 0.2065
✅  New best model saved (val loss 0.2065)


[Train] Epoch 2: 100%|██████████| 625/625 [02:17<00:00,  4.56it/s]


Epoch 2 | train 0.2073 | val 0.2070


...

Epoch 18 | train 0.1191 | val 0.1163
✅  New best model saved (val loss 0.1163)


[Train] Epoch 19: 100%|██████████| 625/625 [02:18<00:00,  4.52it/s]


Epoch 19 | train 0.1094 | val 0.1065
✅  New best model saved (val loss 0.1065)


[Train] Epoch 20: 100%|██████████| 625/625 [02:18<00:00,  4.53it/s]


Epoch 20 | train 0.1084 | val 0.1121


In [8]:
%%time
test_files = list(Path('/kaggle/input/waveform-inversion/test').glob('*.npy'))
len(test_files)

CPU times: user 248 ms, sys: 56.4 ms, total: 305 ms
Wall time: 894 ms


65818

In [9]:
x_cols = [f'x_{i}' for i in range(1, 70, 2)]
fieldnames = ['oid_ypos'] + x_cols
class TestDataset(Dataset):
    def __init__(self, test_files):
        self.test_files = test_files


    def __len__(self):
        return len(self.test_files)


    def __getitem__(self, i):
        test_file = self.test_files[i]

        return np.load(test_file), test_file.stem

ds = TestDataset(test_files)
dl = DataLoader(ds, batch_size=8, num_workers=4, pin_memory=True)

In [10]:
PATH = "/kaggle/working/best_model.pth" 
model.eval()
model.load_state_dict(torch.load(PATH, weights_only=True))

with open('submission.csv', 'wt', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    for inputs, oids_test in tqdm(dl, desc='test'):
        inputs = inputs.to(device)
        with torch.inference_mode():
            outputs = model(inputs)
            
        preds_norm = outputs[:, 0]
        preds_true = 0.5 * (preds_norm + 1.0) * (label_max - label_min) + label_min
        y_preds    = preds_true.cpu().numpy()
        
        for y_pred, oid_test in zip(y_preds, oids_test):
            for y_pos in range(70):
                row = dict(
                    zip(
                        x_cols,
                        [y_pred[y_pos, x_pos] for x_pos in range(1, 70, 2)]
                    )
                )
                row['oid_ypos'] = f"{oid_test}_y_{y_pos}"
            
                writer.writerow(row)

test: 100%|██████████| 8228/8228 [12:38<00:00, 10.85it/s]
